In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import RAKE
import operator

In [ ]:
pip install python-rake==1.4.4

  Preparing metadata (setup.py) ... done
  Created wheel for python-rake: filename=python_rake-1.4.4-py3-none-any.whl size=13437 sha256=b2ada8cb18d1d32550987a1006a89bc312249e013dce4b4f2f4a3aa66db80261
  Stored in directory: /root/.cache/pip/wheels/a5/8f/00/bf4e7adcb63ddd1a00f00ef3ce57ecdb35cb243f18cc894a47
Successfully built python-rake


In [ ]:
stopwords = "/content/stopwords-id.txt"
rake_obj = RAKE.Rake(stopwords)

In [ ]:
def sort_keywords(tup):
  tup.sort(key=lambda x:x[1])
  return tup

In [ ]:
def crawl_komiku(limit, jenis = None, genre = None):
    if genre == None:
      url = 'https://komiku.id/' + jenis.lower()
    elif jenis == None:
      url = 'https://komiku.id/' + genre.lower()
    else:
      url = 'https://komiku.id/manga?orderby=&category_name=' + jenis.lower() + '&orderby=&genre=' + genre.lower()

    titles_limit = limit
    print(f'URL:  {url}')

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            manga_items = soup.find_all('div', class_='bge')

            for i, manga_item in enumerate(manga_items):
                if i == titles_limit:
                    break

                title = manga_item.find('div', class_='kan').h3.text.strip()
                genre = manga_item.find('div', class_='tpe1_inf').b.text.strip()
                update_status = manga_item.find('span', class_='up').text.strip()

                # Get URL detail untuk tiap manga
                detail = manga_item.find('div', class_='kan').find('a').get('href')

                response_details = requests.get(detail)
                soup_detail = BeautifulSoup(response_details.text, 'html.parser')

                manga_details = soup_detail.find_all('main', class_='perapih')

                print(f'Title: {title}')
                print(f'Genre: {genre}')
                # print(f'First Chapter: {first_chapter}')
                # print(f'Latest Chapter: {latest_chapter}')
                print(f'Detail: {detail}')


                for i, manga_detail_item in enumerate(manga_details):
                  sinopsis_element = manga_detail_item.find('p', class_='desc')
                  if sinopsis_element:
                      sinopsis = sinopsis_element.text.strip()
                      print(f'Sinopsis: {sinopsis}')
                  else:
                      print('Sinopsis tidak ditemukan')

                # Get keywords
                sinopsis_keyword = sort_keywords(rake_obj.run(sinopsis))[-3:]
                print('Keyword penting:')
                for word, value in sinopsis_keyword:
                    print(f'{word}: {value}')

                print('=' *50)

        else:
            if response.status_code == 404 :
                print('Page not found')
            print(f'Failed to retrieve the page. Status code: {response.status_code}')

    except Exception as e:
        print(f'Error: {e}')

In [ ]:
# # Main
# limit = int(input("Masukkan limit hasil yang ingin dikeluarkan: "))
# jenis = input("Masukkan jenis komik yang ingin dicari: ")
# genre = input("Masukkan genre yang ingin dicari: ")

crawl_komiku(3, 'manhua', 'action')

URL:  https://komiku.id/manga?orderby=&category_name=manhua&orderby=&genre=action
Title: Urban God of Medicine
Genre (from main page): Manhua Fantasi
Genre (from detail page): ['Action', 'Drama', 'Fantasy', 'Shounen']
Detail: https://komiku.id/manga/urban-god-of-medicine/
Sinopsis: Lima tahun lalu, keluarga Ye musnah dan sampah tak berguna Ye Chen menghilang secara misterius. Lima tahun kemudian, sampah keluarga Ye itu telah kembali! Kembali dengan kekuatan kultivasi yang bisa menjatuhkan para dewa dan mengguncang bumi! Kali ini, Ye Chen akan membuat langit gemetar kagum! Bumi akan gemetar ketakutannya! Dia akan memerintah atas segalanya dan naik ke puncak dunia!
Keyword penting:
menjatuhkan: 1.0
dewa: 1.0
memerintah: 1.0
Title: Magic Emperor
Genre (from main page): Manhua Fantasi
Genre (from detail page): ['Action', 'Adventure', 'Fantasy', 'Magic', 'Martial Arts', 'Supernatural']
Detail: https://komiku.id/manga/magic-emperor/
Sinopsis: “Zhuo Yifan adalah seorang kaisar sihir atau bisa

In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_komiku(limit, jenis=None, genre=None):
    if genre == None:
        url = 'https://komiku.id/' + jenis.lower()
    elif jenis == None:
        url = 'https://komiku.id/' + genre.lower()
    else:
        url = 'https://komiku.id/manga?orderby=&category_name=' + jenis.lower() + '&orderby=&genre=' + genre.lower()

    titles_limit = limit
    print(f'URL:  {url}')

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            manga_items = soup.find_all('div', class_='bge')

            for i, manga_item in enumerate(manga_items):
                if i == titles_limit:
                    break

                title = manga_item.find('div', class_='kan').h3.text.strip()
                update_status = manga_item.find('span', class_='up').text.strip()

                # Get URL detail untuk tiap manga
                detail = manga_item.find('div', class_='kan').find('a').get('href')

                response_details = requests.get(detail)
                soup_detail = BeautifulSoup(response_details.text, 'html.parser')

                manga_details = soup_detail.find_all('main', class_='perapih')

                # Extract genre from detail page
                genres_from_detail = extract_genre_from_detail(soup_detail)
                status_detail_page = get_status(soup_detail)

                # Extract the last chapter number
                last_chapter_number = extract_last_chapter_number(soup_detail)

                # Extract the synopsis
                sinopsis = soup_detail.find('p', class_='desc').text.strip()

                print(f'Detail Link: {detail}')
                print(f'Title: {title}')
                print(f'Genre (from detail page): {genres_from_detail}')
                print(f'Status (from detail page): {status_detail_page}')
                print(f'Last Chapter Number: {last_chapter_number}')
                print(f'Sinopsis: {sinopsis}')

                print('=' * 50)

        else:
            if response.status_code == 404:
                print('Page not found')
            print(f'Failed to retrieve the page. Status code: {response.status_code}')

    except Exception as e:
        print(f'Error: {e}')

def extract_last_chapter_number(soup_detail):
    try:
        last_chapter_element = soup_detail.find('td', class_='judulseries').find('span')
        if last_chapter_element:
            last_chapter_text = last_chapter_element.text.strip()
            last_chapter_number = last_chapter_text.split()[-1]
            return last_chapter_number
        else:
            return 'Chapter number not found'
    except Exception as e:
        print(f'Error extracting last chapter number: {e}')
        return 'Chapter number not found'

# Rest of the code remains unchanged

# Example usage
crawl_komiku(3, 'manhua', 'action')

URL:  https://komiku.id/manga?orderby=&category_name=manhua&orderby=&genre=action
Detail Link: https://komiku.id/manga/urban-god-of-medicine/
Title: Urban God of Medicine
Genre (from detail page): ['Action', 'Drama', 'Fantasy', 'Shounen']
Status (from detail page): Ongoing
Last Chapter Number: 80
Sinopsis: Lima tahun lalu, keluarga Ye musnah dan sampah tak berguna Ye Chen menghilang secara misterius. Lima tahun kemudian, sampah keluarga Ye itu telah kembali! Kembali dengan kekuatan kultivasi yang bisa menjatuhkan para dewa dan mengguncang bumi! Kali ini, Ye Chen akan membuat langit gemetar kagum! Bumi akan gemetar ketakutannya! Dia akan memerintah atas segalanya dan naik ke puncak dunia!
Detail Link: https://komiku.id/manga/magic-emperor/
Title: Magic Emperor
Genre (from detail page): ['Action', 'Adventure', 'Fantasy', 'Magic', 'Martial Arts', 'Supernatural']
Status (from detail page): Ongoing
Last Chapter Number: 477
Sinopsis: “Zhuo Yifan adalah seorang kaisar sihir atau bisa di pangg

In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_komiku():
    base_url = 'https://komiku.id/manga/page/'

    try:
        curr_page = 1

        while True:
          url = f'{base_url}{curr_page}'
          print(f'Processing page {curr_page}: {url}')

          response = requests.get(url)
          if response.status_code == 200:
              soup = BeautifulSoup(response.text, 'html.parser')
              manga_items = soup.find_all('div', class_='bge')

              for i, manga_item in enumerate(manga_items):
                  title = manga_item.find('div', class_='kan').h3.text.strip()
                  update_status = manga_item.find('span', class_='up').text.strip()

                  # Get URL detail untuk tiap manga
                  detail = manga_item.find('div', class_='kan').find('a').get('href')

                  response_details = requests.get(detail)
                  soup_detail = BeautifulSoup(response_details.text, 'html.parser')

                  manga_details = soup_detail.find_all('main', class_='perapih')

                  # Extract genre from detail page
                  genres_from_detail = extract_genre_from_detail(soup_detail)
                  status_detail_page = get_status(soup_detail)

                  # Extract the last chapter number
                  last_chapter_number = extract_last_chapter_number(soup_detail)

                  # Extract the synopsis
                  sinopsis = soup_detail.find('p', class_='desc').text.strip()

                  print(f'Detail Link: {detail}')
                  print(f'Title: {title}')
                  print(f'Genre (from detail page): {genres_from_detail}')
                  print(f'Status (from detail page): {status_detail_page}')
                  print(f'Last Chapter Number: {last_chapter_number}')
                  print(f'Sinopsis: {sinopsis}')

                  print('=' * 50)

          else:
              if response.status_code == 404:
                  print('Page not found')
              print(f'Failed to retrieve the page. Status code: {response.status_code}')

          curr_page += 1

    except Exception as e:
        print(f'Error: {e}')

def extract_last_chapter_number(soup_detail):
    try:
        last_chapter_element = soup_detail.find('td', class_='judulseries').find('span')
        if last_chapter_element:
            last_chapter_text = last_chapter_element.text.strip()
            last_chapter_number = last_chapter_text.split()[-1]
            return last_chapter_number
        else:
            return 'Chapter number not found'
    except Exception as e:
        print(f'Error extracting last chapter number: {e}')
        return 'Chapter number not found'

def get_status(soup):
    status_element = soup.find('tr', string='Status')
    if status_element:
        status = status_element.find_next('td').text.strip()
        return status
    else:
        return 'Status not found'

def extract_genre_from_detail(soup_detail):
    try:
        genre_elements = soup_detail.select('ul.genre li.genre a')
        genres = [genre.text.strip() for genre in genre_elements]
        return genres
    except Exception as e:
        print(f'Error extracting genres from detail page: {e}')
        return None

# Example usage
crawl_komiku()

Processing page 1: https://komiku.id/manga/page/1
Detail Link: https://komiku.id/manga/disastrous-necromancer/
Title: Disastrous Necromancer
Genre (from detail page): ['Action', 'Adventure', 'Fantasy', 'Magic', 'Supernatural']
Status (from detail page): Status not found
Last Chapter Number: 41
Sinopsis: Permainan telah menjadi kenyataan dan tatanan dunia terbalik. Satu-satunya pilihan adalah mendapatkan kelas dan naik level untuk menjadi lebih kuat! Untuk menjadi puncak dunia! Pada hari mendapatkan kelas, Lin Mo yu mendapatkan satu-satunya kelas tersembunyi, Necromancer. Sejak hari itu dan seterusnya, Lin Mo yu tidak akan mati kecuali semua panggilannya dimusnahkan. Saya duduk di singgasana tulang dan menjalin jalan antara hidup dan mati. Aku adalah bencana!
Detail Link: https://komiku.id/manga/i-can-snatch-999-types-of-abilities/
Title: I Can Snatch 999 Types of Abilities
Genre (from detail page): ['Action', 'Fantasy', 'Shounen', 'Supernatural']
Status (from detail page): Status not f

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def crawl_komiku():
    base_url = 'https://komiku.id/manga/page/'
    manga_list = []

    try:
        curr_page = 1
        manga_count = 0

        while curr_page <= 258:
            url = f'{base_url}{curr_page}'
            print(f'Processing page {curr_page}: {url}')

            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                manga_items = soup.find_all('div', class_='bge')

                for i, manga_item in enumerate(manga_items):
                    try:
                        title = manga_item.find('div', class_='kan').h3.text.strip()
                        update_status = manga_item.find('span', class_='up').text.strip()

                        # Get URL detail untuk tiap manga
                        detail = manga_item.find('div', class_='kan').find('a').get('href')

                        response_details = requests.get(detail)
                        soup_detail = BeautifulSoup(response_details.text, 'html.parser')

                        manga_details = soup_detail.find_all('main', class_='perapih')

                        # Extract genre from detail page
                        genres_from_detail = extract_genre_from_detail(soup_detail)
                        status_detail_page = get_status(soup_detail)

                        # Extract the last chapter number
                        last_chapter_number = extract_last_chapter_number(soup_detail)

                        # Extract jenis komik
                        jenis_komik = extract_jenis_komik(soup_detail)


                        # Extract and clean sinopsis
                        raw_sinopsis = soup_detail.find('p', class_='desc').text.strip()
                        sinopsis = clean_sinopsis(raw_sinopsis)

                        manga_list.append({
                            'Detail Link': detail,
                            'Title': title,
                            'Jenis komik': jenis_komik,
                            'Genre (from detail page)': genres_from_detail,
                            'Status (from detail page)': status_detail_page,
                            'Last Chapter Number': last_chapter_number,
                            'Sinopsis': sinopsis
                        })

                        print(f'Detail Link: {detail}')
                        print(f'Title: {title}')
                        print(f'Jenis komik:  {jenis_komik}')
                        print(f'Genre (from detail page): {genres_from_detail}')
                        print(f'Status (from detail page): {status_detail_page}')
                        print(f'Last Chapter Number: {last_chapter_number}')
                        print(f'Sinopsis: {sinopsis}')

                        print('=' * 50)
                        manga_count += 1

                    except Exception as manga_error:
                        print(f'Error processing manga: {title}. Skipping to the next manga. Error: {manga_error}')

            else:
                if response.status_code == 404:
                    print('Page not found')
                print(f'Failed to retrieve the page. Status code: {response.status_code}')

            curr_page += 1

        print('Manga count:', manga_count)
        return manga_list

    except Exception as e:
        print(f'Error: {e}')

def extract_last_chapter_number(soup_detail):
    try:
        last_chapter_element = soup_detail.find('td', class_='judulseries').find('span')
        if last_chapter_element:
            last_chapter_text = last_chapter_element.text.strip()
            # Use regular expression to find all numbers in the text
            numbers = re.findall(r'\d+', last_chapter_text)
            if numbers:
                # Return the last number found, assuming it's the chapter number
                return int(numbers[-1])
            else:
                return 'Chapter number not found'
        else:
            return 'Chapter number not found'
    except Exception as e:
        print(f'Error extracting last chapter number: {e}')
        return 'Chapter number not found'

def extract_jenis_komik(soup_detail):
    inf_table = soup_detail.find('table', class_='inftable')

    if inf_table:
        tag = inf_table.find('b')

        if tag:
            jenis_komik = tag.text.strip()
            return jenis_komik
        else:
            return None
    else:
        return None

def get_status(soup):
    status_element = soup.find('td', string='Status')
    if status_element:
        status = status_element.find_next('td').text.strip()
        return status
    else:
        return 'Status not found'

def extract_genre_from_detail(soup_detail):
    try:
        genre_elements = soup_detail.select('ul.genre li.genre a')
        genres = [genre.text.strip() for genre in genre_elements]
        return genres
    except Exception as e:
        print(f'Error extracting genres from detail page: {e}')
        return None

def clean_sinopsis(raw_sinopsis):
    cleaned_sinopsis = raw_sinopsis.strip().strip('"')

    # Remove extra spaces and line breaks
    cleaned_sinopsis = ' '.join(cleaned_sinopsis.split())

    return cleaned_sinopsis

In [ ]:
manga_list = crawl_komiku()

Streaming output truncated to the last 5000 lines.
Status (from detail page): Ongoing
Last Chapter Number: 15
Sinopsis: Manga Junk the Black Shadow bercerita tentang Memulai kehidupan muridnya, Junk yang pemberontak bosan memenuhi harapan cerah ayahnya Seorang putra seorang paladin mengambil tempat ayahnya di Akademi Sihir Granbell dengan jenis sihir yang langka Pelajaran dinamis sihir mulai di Avista ketika seorang anak laki-laki akhirnya keluar dari bayangan ayahnya Ayah Junk adalah salah satu dari dua belas jenderal yang memimpin para penyihir, Paladin. Meskipun demikian, sebagai siswa yang miskin, ia adalah kebalikan dari ayahnya. Manga fantasi seorang yang kurang berprestasi di jalan yang benar dimulai!
Jumlah Pembaca: 12998
Detail Link: https://komiku.id/manga/the-supreme-urban-heavenly/
Title: The Supreme Urban Heavenly
Jenis komik:  Manhua
Genre (from detail page): ['Action', 'Demons', 'Fantasy', 'Harem', 'Magic', 'Martial Arts', 'Supernatural']
Status (from detail page): Ongoi

KeyboardInterrupt: ignored

In [ ]:
 for manga in manga_list:
        print(manga)

{'Detail Link': 'https://komiku.id/manga/kimetsu-no-yaiba-indonesia/', 'Title': 'Kimetsu no Yaiba', 'Jenis komik': 'Manga', 'Genre (from detail page)': ['Action', 'Adventure', 'Comedy', 'Drama', 'Historical', 'Martial Arts', 'Supernatural', 'Tragedy'], 'Status (from detail page)': 'End', 'Last Chapter Number': 205, 'Sinopsis': 'Manga Kimetsu no Yaiba merupakan komik karangan Koyoharu Gotouge yang terbit pada tanggal 15 Februari 2016 pada majalan Shounen JUMP, bercerita tentang Sejak zaman kuno, banyak rumor tentang setan pemakan manusia yang bersembunyi di hutan. Karena itu, penduduk kota setempat tidak pernah berani keluar di malam hari. Legenda mengatakan bahwa pembunuh iblis juga berkeliaran di malam hari, memburu iblis yang haus darah ini. Sejak kematian ayahnya, Tanjirou telah mengambilnya sendiri untuk mendukung ibu dan lima saudara kandungnya. Meskipun hidup mereka mungkin diperkeras oleh tragedi, mereka telah menemukan kebahagiaan. Tapi kehangatan sesaat itu hancur pada suatu h

In [ ]:
import csv

if manga_list:
    csv_filename = 'manga_list.csv'

    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        # Define CSV header
        fieldnames = ['Detail Link', 'Title', 'Jenis komik', 'Genre (from detail page)',
                      'Status (from detail page)', 'Last Chapter Number', 'Sinopsis','Jumlah Pembaca']

        # Create a CSV writer
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write manga details to the CSV file
        for manga in manga_list:
            writer.writerow(manga)

    print(f'Manga list exported to {csv_filename}')
else:
    print('No manga data to export.')

Manga list exported to manga_list.csv
